In [ ]:
dataset = "Lung"
All_h5ad = "/home/krushna/Documents/Data_integration/SCRNA_Datasets/All_h5ad/"
methods_h5ad = '/home/krushna/Documents/Data_integration/imapOthermethods/'

In [ ]:
our_method_emb_dic = {'Immune_Human' : 'Immune_Human_case2_latent_matrix_c_300',
                 'Lung' : 'Lung_case2_latent_matrix_c_300',
                 'Pancreas' : 'Pancreas_case2_latent_matrix_c_250',
                 'Human_Retina': "retina_case2_latent_matrix_c_300"    
                }   
scdreamer_plus_emb_dic = {
                'Immune_Human' : 'Immune_Humanlatent_matrix_c299_v2_crossEv2',
                 'Lung' : 'Lunglatent_matrix_c299_v2_crossEv2',
                 'Pancreas' : 'Pancreaslatent_matrix_c299_v2_crossEv2',
                 'Human_Retina': "Human_Retinalatent_matrix_c299_v2_crossEv2",
                 'Human_Mouse' : 'Human_Mousenewlatent_matrix_c300_scdreamer++'
}

insct_emb_dic = {'Immune_Human' : 'Immune_tnn_embeddings_1510',
             'Lung' : 'Lung_tnn_embeddings_1510',
             'Pancreas' : 'Pancreas_tnn_embeddings_1510',
             'Tabula_Muris' : "tabular_muris_tnn_embeddings_0211",
             'Human_Mouse' : '',
             'Human_Retina': ""    
}   
  
batch_key_dic = {'Immune_Human' : 'batch',
                 'Lung' : 'batch',
                 'Pancreas' : 'tech',
                 'Tabula_Muris' : "batch",
                 'Human_Mouse' : 'batch',
                 'Human_Retina': "Batch"
                }
cell_type_key_dic = {'Immune_Human' : 'final_annotation',
                 'Lung' : 'cell_type',
                 'Pancreas' : 'celltype',
                 "Tabula_Muris" : "cell_ontology_class",
                 'Human_Mouse' : "celltype",
                 "Human_Retina":"Subcluster"
                    } 
insct_emb  = insct_emb_dic[dataset]
our_method_emb = our_method_emb_dic[dataset]
batch_key = batch_key_dic[dataset]
cell_type_key = cell_type_key_dic[dataset]
paths = {
    'scanorama' : "scanorama/scanorama-"+dataset+".h5ad", #adata
    'scvi' : "scvi/scvi-"+dataset+".h5ad", #adata
    'harmony' : "harmony/harmony-"+dataset+".h5ad", #adata
    'seurat' : "seurat/seurat-"+dataset+".h5ad",
    'bbknn' : "bbknn/bbknn-"+dataset+'.h5ad',
    'FastMnn' : "fastmnn/"+dataset+'.h5ad',
    'scGen' : "scGen/scGen-"+dataset+'.h5ad',
    'scanvi' : "scanvi/scanvi-"+dataset+'.h5ad',
    'imap' : "imap/imap-"+dataset+'.h5ad',
    'liger' : "/home/krushna/Documents/Data_integration/Othermethods/liger/embeddings/" + dataset + '.csv',
    'our_method' : "/home/krushna/Documents/Data_integration/Ajita_embeddings/checking_other_cases/final/" + our_method_emb +'.csv',
    'insct' : "/home/krushna/Documents/Data_integration/Ajita_embeddings/non zero kl/" + insct_emb +'.csv',
    'scdreamer_plus' : '/home/krushna/Documents/Data_integration/Ajita_embeddings/scDREAMER++/'+scdreamer_plus_emb_dic[dataset]+'.csv'
}



In [ ]:
import scanpy as sc
import numpy as np

#  R

In [ ]:
import anndata2ri
anndata2ri.activate()

#Loading the rpy2 extension enables cell magic to be used
#This runs R code in jupyter notebook cells
%load_ext rpy2.ipython

sc.settings.verbosity = 3
sc.logging.print_versions()

In [ ]:
%%R
#' @param data: cells * features matrix
#' @param cell_type: named vector
#' @param batch: named vector
#' @param k: number of NNs to classify negative cells

options(stringsAsFactors = FALSE)
require(RcppAnnoy)
require(ggplot2)
require(grid)
require(ggpubr)

color_used <- c(
  "cyan4",      "skyblue3",   "darkolivegreen3",   "lightpink",   "darkmagenta",   "brown",   "blueviolet", "bisque4",  "deeppink3",       "darkkhaki",      
  "dodgerblue4",     "goldenrod4",            "gainsboro",       "firebrick4",      "cadetblue3",
  "greenyellow",     "gray6",           "coral2",                     "yellow4",         
         "darkgoldenrod3",  "navy",            "deepskyblue3","antiquewhite3"
)

benchmark_idx = function(data, cell_type, batch, k = 20, k2 = 100, pos_thr = 3, 
                         plot = FALSE, prefix = '', 
                         color_batch = NULL, 
                         color_cell_type = NULL, 
                         color_index = NULL, 
                         point_size = 1){
    bench_list = list()
    
    falseRates = c()
    tmp_c = table(cell_type)
    
    tmp_b = table(cell_type, batch)
    unique_batch = unique(batch)
    
    
    
    vector_size = dim(data)[2]
    a <- new(AnnoyEuclidean, vector_size)
    a$setSeed(1)
    for(i in 1:dim(data)[1]){
        a$addItem(i-1, data[i,])
    }
    a$build(50)
    
    ks = c()
    for(i in 1:dim(data)[1]){
        k_ = min(k, tmp_c[cell_type[i]])
        ks = c(ks, k_)
        NNs = a$getNNsByItem(i-1, k_+1)[2:(k_+1)] + 1
        
        c_NN = cell_type[NNs]
        false_rate = sum(c_NN != cell_type[i]) / k_
        falseRates = c(falseRates, false_rate)
    }
    
    bench_list[['falseRates']] = falseRates
    bench_list[['NegativeCells']] = falseRates >= 0.5
    bench_list[['NumberOfNegativeCells']] = sum(falseRates >= 0.5)
#     message('Number of negative cells: ', sum(falseRates >= 0.5))
    
    
    ######### discriminate TRUE POSITIVE CELLS
    truePositive = c()
    
    tmp_b_prob = t(apply(tmp_b, 1, function(x) x/sum(x)))
                       
    for(i in 1:dim(data)[1]){
        cur = Inf
        if(falseRates[i] < 0.5){
            k_2 = min(k2, tmp_c[cell_type[i]])
            NNs_ = a$getNNsByItem(i-1, k_2) + 1
            NNs_c = cell_type[i]
            NNs_i = NNs_c == cell_type[i]
            NNs_ = NNs_[NNs_i]
            k_2 = length(NNs_)
            
            NNs_b = batch[NNs_]
            
            b_prob_ = tmp_b_prob[cell_type[i],]
            
            max_b = 0
            for(b in unique_batch){
                if(b_prob_[b] > 0 & b_prob_[b] < 1){
                    p_b = sum(NNs_b == b)
                    sd_b = abs(p_b - k_2 * b_prob_[b]) / sqrt(k_2 * b_prob_[b] * (1-b_prob_[b]))
                    max_b = max(max_b, sd_b)
                }
            }
            cur = max_b
        }
        
        truePositive = c(truePositive, cur)
    }
    
    bench_list[['truePositiveRate']] = truePositive
    bench_list[['truePositiveCells']] = truePositive <= pos_thr 
    bench_list[['NumberOfTruePositiveCells']] = sum(truePositive <= pos_thr)
#     message('Number of true positive cells: ', sum(truePositive <= pos_thr))
           
    labels = rep('Others', length(batch))
    labels[falseRates >= 0.5] = 'Negative cells'
    labels[truePositive <= pos_thr] = 'True positive cells'
    bench_list[['labels']] = labels
    
    if(plot){
        n = length(batch)
        ord = sample(n,n,replace=F)
        
        labels = rep('Others', length(ord))
        labels[falseRates[ord] >= 0.5] = 'Negative cells'
        labels[truePositive[ord] <= pos_thr] = 'True positive cells'
        df = data.frame(
            x = data[ord, 1],
            y = data[ord, 2],
            labels = labels
        )
        
        if(is.null(color_index)){
            color_index = color_used[c(1,9,19)]
        }
        #### plot cell type
        pdf(paste0(prefix, '_index.pdf'), height = 6, width = 8)
        p = df %>% ggplot() + geom_point(aes(x=x, y=y, color=labels), size = point_size, alpha = 1) + theme_pubr() +
            scale_color_manual(values = color_index, name = 'scBM') +
            xlab('UMAP 1') + ylab('UMAP 2') + theme(legend.position = 'right') +
            guides(colour = guide_legend(override.aes = list(size=5))) +
            scale_size_manual(values = .1)
        print(p)
        dev.off()
        
        pdf(paste0(prefix, 'true_positive_cells.pdf'), height = 6, width = 8)
        p = ggplot() + geom_point(aes(x=data[ord,1], y=data[ord,2], color=truePositive[ord] <= pos_thr), size = point_size, alpha = 1) + theme_pubr() +
            scale_color_manual(values = c('red', 'blue'), name = 'True Positive') +
            xlab('UMAP 1') + ylab('UMAP 2') + theme(legend.position = 'right') +
            guides(colour = guide_legend(override.aes = list(size=5))) +
            scale_size_manual(values = .1)
        print(p)
        dev.off()
        
        ### Plot cell type
        if(is.null(color_cell_type)){
            no_of_markers <- length(unique(cell_type))
            if (no_of_markers > length(color_used)) {
                color_cell_type <- colorRampPalette(brewer.pal(9, "Set1"))(no_of_markers+1)
            }
            else {
                color_cell_type <- color_used
            }
        }
        pdf(paste0(prefix,'cell_types.pdf'), height = 6, width = 8)
        p = ggplot() + geom_point(aes(x=data[ord,1], y=data[ord,2], color=cell_type[ord]), size = point_size, alpha = 1) + theme_pubr() +
            scale_color_manual(values = color_cell_type, name = 'Cell type') +
            xlab('UMAP 1') + ylab('UMAP 2') + theme(legend.position = 'right') +
            guides(colour = guide_legend(override.aes = list(size=5))) +
            scale_size_manual(values = .1)
        print(p)
        dev.off()
    
#          theme(
#                 axis.text.y = element_blank(),
#                 axis.text.x = element_blank(),
#                 legend.position = 'right',
#                 axis.ticks.x=element_blank(),
#                 axis.ticks.y=element_blank()
#             )
        ### Plot cell type
        if(is.null(color_batch)){
            color_batch = rev(color_used)
        }
        pdf(paste0(prefix,'batches.pdf'), height = 6, width = 8)
        
        p = ggplot() + geom_point(aes(x=data[ord,1], y=data[ord,2], color=batch[ord]), size = point_size, alpha = 1) + theme_pubr() +
            scale_color_manual(values = color_batch, name = batch_key) +
            xlab('UMAP 1') + ylab('UMAP 2') + theme(legend.position = 'right') +
            guides(colour = guide_legend(override.aes = list(size=5))) +
            scale_size_manual(values = .1)
        print(p)
        dev.off()
    }
    
    return(bench_list)
}
 
benchmark_idx_new = function(data, cell_type, batch, k = 20, k2 = 100, pos_thr = 1, 
                         plot = FALSE, prefix = '', 
                         color_batch = NULL, 
                         color_cell_type = NULL, 
                         color_index = NULL, 
                         point_size = 1){
    bench_list = list()
    
    falseRates = c()
    tmp_c = table(cell_type)
    
    tmp_b = table(cell_type, batch)
    unique_batch = unique(batch)
    
    
    
    vector_size = dim(data)[2]
    a <- new(AnnoyEuclidean, vector_size)
    a$setSeed(1)
    for(i in 1:dim(data)[1]){
        a$addItem(i-1, data[i,])
    }
    a$build(50)
    
    ks = c()
    for(i in 1:dim(data)[1]){
        k_ = min(k, tmp_c[cell_type[i]])
        ks = c(ks, k_)
        NNs = a$getNNsByItem(i-1, k_+1)[2:(k_+1)] + 1
        
        c_NN = cell_type[NNs]
        false_rate = sum(c_NN != cell_type[i]) / k_
        falseRates = c(falseRates, false_rate)
    }
    
    bench_list[['falseRates']] = falseRates
    bench_list[['NegativeCells']] = falseRates >= 0.5
    bench_list[['NumberOfNegativeCells']] = sum(falseRates >= 0.5)
#     message('Number of negative cells: ', sum(falseRates >= 0.5))
    
    
    ######### discriminate TRUE POSITIVE CELLS
    truePositive = c()
    
    tmp_b_prob = t(apply(tmp_b, 1, function(x) x/sum(x)))
                       
    for(i in 1:dim(data)[1]){
        sd_b = -Inf
        if(falseRates[i] < 0.5){
            k_2 = min(k2, tmp_c[cell_type[i]])
            NNs_ = a$getNNsByItem(i-1, k_2) + 1
            NNs_c = cell_type[i]
            b = batch[i]
            k_2 = length(NNs_)
            
            NNs_b = batch[NNs_]
            
            b_prob_ = tmp_b_prob[cell_type[i],]
            
            p_b = sum(NNs_b == b)
            
            sd_b = (p_b - k_2 * b_prob_[b]) / sqrt(k_2 * b_prob_[b] * (1-b_prob_[b]))
            
        }
        
        truePositive = c(truePositive, sd_b)
    }
    
    bench_list[['truePositiveRate']] = truePositive
    bench_list[['truePositiveCells']] = truePositive >= pos_thr 
    bench_list[['NumberOfTruePositiveCells']] = sum(truePositive >= pos_thr)
#     message('Number of true positive cells: ', sum(truePositive >= pos_thr))
           
    labels = rep('Others', length(batch))
    labels[falseRates >= 0.5] = 'Negative cells'
    labels[truePositive <= pos_thr] = 'True positive cells'
    bench_list[['labels']] = labels
    
    
    return(bench_list)
}                        

# scanorama

In [ ]:
adata =sc.read_h5ad(paths['scanorama'])
sc.pp.neighbors(adata, use_rep='X_emb')  # use_rep = 'final_embeddings'
sc.tl.umap(adata)
sc.pl.umap(adata, color=cell_type_key, frameon=False)
sc.pl.umap(adata, color=batch_key, frameon=False)

In [ ]:
batch = adata.obs[batch_key].to_numpy()
cell_type = adata.obs[cell_type_key].to_numpy()
data_mat = adata.obsm['X_emb']
data_mat_umap = adata.obsm['X_umap']

In [ ]:
%%R --i data_mat --i batch --i cell_type --i data_mat_umap
print(dim(data_mat))
print(length(batch))
print(length(cell_type))
bench_list <- benchmark_idx(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
print(sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]])))
print(sprintf("True possitive percent : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent umap : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))


bench_list <- benchmark_idx_new(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx_new(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

# Scvi

In [ ]:
adata =sc.read_h5ad(paths['scvi'])
sc.pp.neighbors(adata, use_rep='X_scVI')  # use_rep = 'final_embeddings'
sc.tl.umap(adata)
sc.pl.umap(adata, color=cell_type_key, frameon=False)
sc.pl.umap(adata, color=batch_key, frameon=False)

In [ ]:
batch = adata.obs[batch_key].to_numpy()
cell_type = adata.obs[cell_type_key].to_numpy()
data_mat = adata.obsm['X_scVI']
data_mat_umap = adata.obsm['X_umap']

In [ ]:
%%R --i data_mat --i batch --i cell_type --i data_mat_umap
print(dim(data_mat))
print(length(batch))
print(length(cell_type))
bench_list <- benchmark_idx(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
print(sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]])))
print(sprintf("True possitive percent : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent umap : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))


bench_list <- benchmark_idx_new(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx_new(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

# harmony

In [ ]:
adata =sc.read_h5ad(paths['harmony'])

In [ ]:
sc.pp.neighbors(adata,use_rep='X_harmony')  # use_rep = 'final_embeddings'
sc.tl.umap(adata)
sc.pl.umap(adata, color=cell_type_key, frameon=False)
sc.pl.umap(adata, color=batch_key, frameon=False)

In [ ]:
batch = adata.obs[batch_key].to_numpy()
cell_type = adata.obs[cell_type_key].to_numpy()
data_mat = adata.obsm['X_harmony']
data_mat_umap = adata.obsm['X_umap']

In [ ]:
%%R --i data_mat --i batch --i cell_type --i data_mat_umap
print(dim(data_mat))
print(length(batch))
print(length(cell_type))
bench_list <- benchmark_idx(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
print(sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]])))
print(sprintf("True possitive percent : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent umap : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))


bench_list <- benchmark_idx_new(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx_new(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

# seurat

In [ ]:
adata =sc.read_h5ad(paths['seurat'])

In [ ]:
sc.pp.neighbors(adata)  # use_rep = 'final_embeddings'
sc.tl.umap(adata)
sc.pl.umap(adata, color=cell_type_key, frameon=False)
sc.pl.umap(adata, color=batch_key, frameon=False)

In [ ]:
batch = adata.obs[batch_key].to_numpy()
cell_type = adata.obs[cell_type_key].to_numpy()
data_mat = adata.X.todense()
data_mat_umap = adata.obsm['X_umap']

In [ ]:
%%R --i data_mat --i batch --i cell_type --i data_mat_umap
print(dim(data_mat))
print(length(batch))
print(length(cell_type))
bench_list <- benchmark_idx(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
print(sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]])))
print(sprintf("True possitive percent : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent umap : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))


bench_list <- benchmark_idx_new(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx_new(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

# our method

In [ ]:
import pandas as pd
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
adata =sc.read_h5ad(All_h5ad+dataset+".h5ad")
batch = adata.obs[batch_key].to_numpy()
cell_type = adata.obs[cell_type_key].to_numpy()
data_mat = pd.read_csv(paths['our_method'],sep=',',header=None).values
adata.obsm['X_emb'] = data_mat
sc.pp.neighbors(adata, use_rep = 'X_emb')  # use_rep = 'final_embeddings'
sc.tl.umap(adata)
data_mat_umap = adata.obsm['X_umap']
sc.settings.figdir = "/home/krushna/Documents/Data_integration/Figures/positive_vs_true_positive/"
path_to_save_umap = "_ourmethod_" +dataset+"_cell.png"
sc.pl.umap(adata, color=cell_type_key, frameon=False, save = path_to_save_umap)
path_to_save_umap = "_ourmethod_" +dataset+"_batch.png"
sc.pl.umap(adata, color=batch_key, frameon=False, save = path_to_save_umap)

In [ ]:
%%R --i data_mat --i batch --i cell_type --i data_mat_umap
print(dim(data_mat))
print(length(batch))
print(length(cell_type))
bench_list <- benchmark_idx(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
print(sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]])))
print(sprintf("True possitive percent : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent umap : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))


bench_list <- benchmark_idx_new(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx_new(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

In [ ]:
%%R --i data_mat --i batch --i cell_type --i data_mat_umap
print(dim(data_mat))
print(length(batch))
print(length(cell_type))
bench_list <- benchmark_idx(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
print(sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]])))
print(sprintf("True possitive percent : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))


In [ ]:
%%R --i dataset --o bench_list 
saveRDS(bench_list,paste0("/home/krushna/Documents/Data_integration/Othermethods/pos_vs_true_pos/ourmethod_",dataset,"_bench_list.RDS"))
# class(bench_list)

In [ ]:
%%R --i dataset --o bench_list 
bench_list <- readRDS(paste0("/home/krushna/Documents/Data_integration/Othermethods/pos_vs_true_pos/ourmethod_",dataset,"_bench_list.RDS"))

In [ ]:

adata.obs['Evaluation'] = list(bench_list[6])
adata.obs['Evaluation']  = adata.obs['Evaluation'].replace(to_replace = "Others", value = 'Positive cells' )
color_map = {'Negative cells' : 'red', 'Positive cells' : 'lightgreen', 'True positive cells' : 'blue'}
sc.settings.figdir = "/home/krushna/Documents/Data_integration/Figures/positive_vs_true_positive/"
path_to_save_umap = "_ourmethod_" +dataset+".png"
sc.pl.umap(adata, color='Evaluation', frameon=False, palette = color_map, save = path_to_save_umap)

# Insct

In [ ]:
import pandas as pd
adata =sc.read_h5ad(All_h5ad+dataset+".h5ad")
batch = adata.obs[batch_key].to_numpy()
cell_type = adata.obs[cell_type_key].to_numpy()
data_mat = pd.read_csv(paths['insct'],sep=',',header=None).values
adata.obsm['X_emb'] = data_mat
sc.pp.neighbors(adata, use_rep = 'X_emb')  # use_rep = 'final_embeddings'
sc.tl.umap(adata)
data_mat_umap = adata.obsm['X_umap']
sc.pl.umap(adata, color=cell_type_key, frameon=False)
sc.pl.umap(adata, color=batch_key, frameon=False)

In [ ]:
%%R --i data_mat --i batch --i cell_type --i data_mat_umap
print(dim(data_mat))
print(length(batch))
print(length(cell_type))
bench_list <- benchmark_idx(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
print(sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]])))
print(sprintf("True possitive percent : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent umap : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))


bench_list <- benchmark_idx_new(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx_new(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

# scANVI

In [ ]:
adata =sc.read_h5ad(methods_h5ad+paths['scanvi'])
sc.pp.neighbors(adata, use_rep='X_scANVI')  # use_rep = 'final_embeddings'
sc.tl.umap(adata)
sc.pl.umap(adata, color=cell_type_key, frameon=False)
sc.pl.umap(adata, color=batch_key, frameon=False)

In [ ]:
adata

In [ ]:
batch = adata.obs[batch_key].to_numpy()
cell_type = adata.obs[cell_type_key].to_numpy()
data_mat = adata.obsm['X_scANVI']
data_mat_umap = adata.obsm['X_umap']

In [ ]:
%%R --i data_mat --i batch --i cell_type --i data_mat_umap
print(dim(data_mat))
print(length(batch))
print(length(cell_type))
bench_list <- benchmark_idx(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
print(sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]])))
print(sprintf("True possitive percent : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent umap : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))


bench_list <- benchmark_idx_new(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx_new(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

# scGen

In [ ]:
adata =sc.read_h5ad(methods_h5ad+paths['scGen'])

In [ ]:
sc.pp.neighbors(adata)  # use_rep = 'final_embeddings'
sc.tl.umap(adata)
sc.pl.umap(adata, color=cell_type_key, frameon=False)
sc.pl.umap(adata, color=batch_key, frameon=False)

In [ ]:
batch = adata.obs[batch_key].to_numpy()
cell_type = adata.obs[cell_type_key].to_numpy()
data_mat = adata.X
data_mat_umap = adata.obsm['X_umap']

In [ ]:
%%R --i data_mat --i batch --i cell_type --i data_mat_umap
print(dim(data_mat))
print(length(batch))
print(length(cell_type))
bench_list <- benchmark_idx(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
print(sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]])))
print(sprintf("True possitive percent : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent umap : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))


bench_list <- benchmark_idx_new(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx_new(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

# FastMnn

In [ ]:
adata =sc.read_h5ad(methods_h5ad+paths['FastMnn'])

In [ ]:
sc.pp.neighbors(adata)  # use_rep = 'final_embeddings'
sc.tl.umap(adata)
sc.pl.umap(adata, color=cell_type_key, frameon=False)
sc.pl.umap(adata, color=batch_key, frameon=False)

In [ ]:
batch = adata.obs[batch_key].to_numpy()
cell_type = adata.obs[cell_type_key].to_numpy()
data_mat = adata.X
data_mat_umap = adata.obsm['X_umap']

In [ ]:
%%R --i data_mat --i batch --i cell_type --i data_mat_umap
print(dim(data_mat))
print(length(batch))
print(length(cell_type))
bench_list <- benchmark_idx(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
print(sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]])))
print(sprintf("True possitive percent : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent umap : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))


bench_list <- benchmark_idx_new(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx_new(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

# imap

In [ ]:
adata =sc.read_h5ad(methods_h5ad+paths['imap'])

In [ ]:
sc.pp.neighbors(adata)  # use_rep = 'final_embeddings'
sc.tl.umap(adata)
sc.pl.umap(adata, color=cell_type_key, frameon=False)
sc.pl.umap(adata, color=batch_key, frameon=False)

In [ ]:
batch = adata.obs[batch_key].to_numpy()
cell_type = adata.obs[cell_type_key].to_numpy()
data_mat = adata.X
data_mat_umap = adata.obsm['X_umap']

In [ ]:
%%R --i data_mat --i batch --i cell_type --i data_mat_umap
print(dim(data_mat))
print(length(batch))
print(length(cell_type))
bench_list <- benchmark_idx(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
print(sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]])))
print(sprintf("True possitive percent : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent umap : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))


bench_list <- benchmark_idx_new(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx_new(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

# Liger

In [ ]:
import pandas as pd
adata =sc.read_h5ad(All_h5ad+dataset+".h5ad")
batch = adata.obs[batch_key].to_numpy()
cell_type = adata.obs[cell_type_key].to_numpy()
data_mat = pd.read_csv(paths['liger'],sep=',',index_col=0).values
adata.obsm['X_emb'] = data_mat
sc.pp.neighbors(adata, use_rep = 'X_emb')  # use_rep = 'final_embeddings'
sc.tl.umap(adata)
data_mat_umap = adata.obsm['X_umap']
sc.pl.umap(adata, color=cell_type_key, frameon=False)
sc.pl.umap(adata, color=batch_key, frameon=False)

In [ ]:
%%R --i data_mat --i batch --i cell_type --i data_mat_umap
print(dim(data_mat))
print(length(batch))
print(length(cell_type))
bench_list <- benchmark_idx(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
print(sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]])))
print(sprintf("True possitive percent : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent umap : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))


bench_list <- benchmark_idx_new(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx_new(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

# scDREAMER_plus

In [ ]:
import pandas as pd
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
adata =sc.read_h5ad(All_h5ad+dataset+".h5ad")
batch = adata.obs[batch_key].to_numpy()
cell_type = adata.obs[cell_type_key].to_numpy()
data_mat = pd.read_csv(paths['scdreamer_plus'],sep=',',header=None).values
adata.obsm['X_emb'] = data_mat
sc.pp.neighbors(adata, use_rep = 'X_emb')  # use_rep = 'final_embeddings'
sc.tl.umap(adata)
data_mat_umap = adata.obsm['X_umap']
sc.settings.figdir = "/home/krushna/Documents/Data_integration/Figures/positive_vs_true_positive/"
path_to_save_umap = "_scdreamer_plus_" +dataset+"_cell.png"
sc.pl.umap(adata, color=cell_type_key, frameon=False, save = path_to_save_umap)
path_to_save_umap = "_scdreamer_plus_" +dataset+"_batch.png"
sc.pl.umap(adata, color=batch_key, frameon=False, save = path_to_save_umap)

In [ ]:
%%R --i data_mat --i batch --i cell_type --i data_mat_umap
print(dim(data_mat))
print(length(batch))
print(length(cell_type))
bench_list <- benchmark_idx(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
print(sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]])))
print(sprintf("True possitive percent : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent umap : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))


bench_list <- benchmark_idx_new(data_mat, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))

bench_list <- benchmark_idx_new(data_mat_umap, cell_type, batch)
possitive = sum(!bench_list[["NegativeCells"]])
# sprintf("possitive percent : %f",100*possitive/length(bench_list[["NegativeCells"]]))
print(sprintf("True possitive percent new umap : %f",100*bench_list[['NumberOfTruePositiveCells']]/possitive))